# Download/Saving CIFAR-10 images in Inception format
--------------------------------------

In this script, we download the CIFAR-10 images and transform/save them in the Inception Retraining Format.  The end purpose of the files is for re-training the Google Inception tensorflow model to work on the CIFAR-10.

We start by loading the necessary libraries and clearing out any current default computational graph.

In [3]:
import os
import tarfile
import _pickle as cPickle
import numpy as np
import urllib.request
import scipy.misc
from tensorflow.python.framework import ops
ops.reset_default_graph()

Download the data.

In [4]:
cifar_link = 'https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz'
data_dir = 'temp'
if not os.path.isdir(data_dir):
    os.makedirs(data_dir)

# Download tar file
target_file = os.path.join(data_dir, 'cifar-10-python.tar.gz')
if not os.path.isfile(target_file):
    print('CIFAR-10 file not found. Downloading CIFAR data (Size = 163MB)')
    print('This may take a few minutes, please wait.')
    filename, headers = urllib.request.urlretrieve(cifar_link, target_file)

# Extract into memory
tar = tarfile.open(target_file)
tar.extractall(path=data_dir)
tar.close()
objects = ['airplane', 'automobile', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck']

CIFAR-10 file not found. Downloading CIFAR data (Size = 163MB)
This may take a few minutes, please wait.


Next we save the images in the corresponding train or test folders.

In [6]:
# Create train image folders
train_folder = 'train_dir'
if not os.path.isdir(os.path.join(data_dir, train_folder)):
    for i in range(10):
        folder = os.path.join(data_dir, train_folder, objects[i])
        os.makedirs(folder)
# Create test image folders
test_folder = 'validation_dir'
if not os.path.isdir(os.path.join(data_dir, test_folder)):
    for i in range(10):
        folder = os.path.join(data_dir, test_folder, objects[i])
        os.makedirs(folder)

# Extract images accordingly
data_location = os.path.join(data_dir, 'cifar-10-batches-py')
train_names = ['data_batch_' + str(x) for x in range(1,6)]
test_names = ['test_batch']

Next we create a function that will load the images from the files.

In [7]:
def load_batch_from_file(file):
    file_conn = open(file, 'rb')
    image_dictionary = cPickle.load(file_conn, encoding='latin1')
    file_conn.close()
    return image_dictionary

Create a function that will save the images from a dictionary into correctly formatted image files.

In [8]:
def save_images_from_dict(image_dict, folder='data_dir'):
    # image_dict.keys() = 'labels', 'filenames', 'data', 'batch_label'
    for ix, label in enumerate(image_dict['labels']):
        folder_path = os.path.join(data_dir, folder, objects[label])
        filename = image_dict['filenames'][ix]
        #Transform image data
        image_array = image_dict['data'][ix]
        image_array.resize([3, 32, 32])
        # Save image
        output_location = os.path.join(folder_path, filename)
        scipy.misc.imsave(output_location,image_array.transpose())

Next we sort the training and test images into the corresponding folders

In [9]:
# Sort train images
for file in train_names:
    print('Saving images from file: {}'.format(file))
    file_location = os.path.join(data_dir, 'cifar-10-batches-py', file)
    image_dict = load_batch_from_file(file_location)
    save_images_from_dict(image_dict, folder=train_folder)

# Sort test images
for file in test_names:
    print('Saving images from file: {}'.format(file))
    file_location = os.path.join(data_dir, 'cifar-10-batches-py', file)
    image_dict = load_batch_from_file(file_location)
    save_images_from_dict(image_dict, folder=test_folder)

Saving images from file: data_batch_1


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:11: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
  # This is added back by InteractiveShellApp.init_path()


Saving images from file: data_batch_2
Saving images from file: data_batch_3
Saving images from file: data_batch_4
Saving images from file: data_batch_5
Saving images from file: test_batch


Correctly label the images/folders in a text file:

In [10]:
# Create labels file
cifar_labels_file = os.path.join(data_dir,'cifar10_labels.txt')
print('Writing labels file, {}'.format(cifar_labels_file))
with open(cifar_labels_file, 'w') as labels_file:
    for item in objects:
        labels_file.write("{}\n".format(item))

Writing labels file, temp/cifar10_labels.txt


Now we have the images in the following format:

```
-train_dir
  |--airplane
  |--automobile
  |--bird
  |--cat
  |--deer
  |--dog
  |--frog
  |--horse
  |--ship
  |--truck
-validation_dir
  |--airplane
  |--automobile
  |--bird
  |--cat
  |--deer
  |--dog
  |--frog
  |--horse
  |--ship
  |--truck
```

After this is done, we proceed with the [TensorFlow fine-tuning tutorial](https://www.tensorflow.org/tutorials/image_retraining).

https://www.tensorflow.org/tutorials/image_retraining